# Audio Common
This module contains the "common" data to all other modules, like basic types.

## Setup

### Prerequisites
Be sure you've run `install.sh` before running this notebook!

### Settings

In [1]:
#Export
from pathlib import Path
import mimetypes
import torchaudio

### Constants and definitions

In [2]:
#Export
AUDIO_EXTENSIONS = tuple(str.lower(k) for k,v in mimetypes.types_map.items() 
                         if v.startswith('audio/'))

In [3]:
#Export 
def getFastAiWorkingDirectory(folder):
    '''Returns the standard working directory for fast.ai for a secific dataset'''
    path = Path(Path.home()/'.fastai/data/')/folder
    if path.exists: print(f'Working directory: {path}')
    else: print('Missing data folder')
    return path

## AudioData

This is the base class of our audio data. It contains two basic information about the "sound":
* sig: the actual signal
* sr: the sample rate

**IMPORTANT:** the audio signal is expected to be one-dimensional i.e. mono. If you have stereo recordings, you should downsample to mono. Later, we could handle this as a preprocessing step, and/or handle stereo files natively.

In [4]:
#Export
class AudioData:
    '''Holds basic information from audio signal'''
    def __init__(self, sig, sr=16000): 
        self.sig = sig.reshape(-1) # We want single dimension data
        self.sr = sr

    def _repr_html_(self): 
        return f'{self.__str__()}<br />{self.ipy_audio._repr_html_()}'

    def hear(self, title=None):
        if title is not None: print(title)
        display(self.ipy_audio)

    @property
    def ipy_audio(self):
        return Audio(data=self.data.sig, rate=self.data.sr)
        
    @classmethod
    def load(cls, fileName, **kwargs):
        p = Path(fileName)
        if p.exists() & str(p).lower().endswith(AUDIO_EXTENSIONS):
            signal,samplerate = torchaudio.load(str(fileName))
            return AudioData(signal,samplerate)
        raise Exception(f"Error while processing {fileName}: file not found or does not have valid extension: {AUDIO_EXTENSIONS}")

## Tests

VERY rudimentary. Ideally we would have sample data that we knew would fail (e.g. non-audio data, audio data with wrong extensions, stereo samples, etc). 

### Sample data for our tests

In [5]:
from fastai.basics import url2name, datapath4file, untar_data
data_url = 'http://www.openslr.org/resources/45/ST-AEDS-20180100_1-OS'
path = datapath4file(url2name(data_url))
untar_data(data_url, dest = path) 
good_sample = path.ls()[256] # arbitrary choice of file

### Direct torchaudio "test" of sample data

In [6]:
signal,samplerate = torchaudio.load(good_sample)

In [7]:
signal.shape, samplerate

(torch.Size([1, 64640]), 16000)

In [8]:
from IPython.display import Audio
Audio(data=signal,rate=samplerate)

In [9]:
s = AudioData.load(good_sample)
display(s)


AttributeError: 'AudioData' object has no attribute 'data'

### More reasonable tests

In [10]:
def is_mono(a): assert 1 == len(a.sig.shape), "Not single dim"
def is_16kHz(a): assert 16000 == a.sr, "Not 16kHz"
def has_data(a): assert a.sig.shape[0] > 100, "Not more than 100 samples"

In [11]:
allTests = lambda x: [f(x) for f in [is_mono, is_16kHz, has_data]]

In [12]:
def test_AudioData_create_from_audio_file_path(f):
    a = AudioData.load(f)
    allTests(a)
    print(f"{f} passed loading from file")

In [13]:
def test_AudioData_create_from_data(f):
    signal,samplerate = torchaudio.load(f)
    a = AudioData(signal,samplerate)
    allTests(a)
    print(f"{f} passed loading from data")

In [14]:
inps = [good_sample, "badpath"] ## Should have bad_samples too
for inp in inps:
    try:
        test_AudioData_create_from_audio_file_path(inp)
        test_AudioData_create_from_data(inp)
    except Exception as e: print(e)

/home/ste/.fastai/data/ST-AEDS-20180100_1-OS/m0005_us_m0005_00174.wav passed loading from file
/home/ste/.fastai/data/ST-AEDS-20180100_1-OS/m0005_us_m0005_00174.wav passed loading from data
Error while processing badpath: file not found or does not have valid extension: ('.aif', '.aifc', '.aiff', '.au', '.mp2', '.mp3', '.ra', '.snd', '.wav')


<span style="color:red">**Careful - trying to torchaudio.load() a non-audio file breaks the kernel!!**</span> That's why we check the extension in `AudioData.load`.

In [15]:
# cwd = %pwd
# test_AudioData_create_from_data(cwd + "/README.md")

## Export

In [16]:
!python notebook2script.py 08za_AudioCommon.ipynb

Converted 08za_AudioCommon.ipynb to exp/nb_08za.py
